## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\ancya\Documents\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.73,-155.09,80.60,65,75,12.75,broken clouds
1,1,Avarua,CK,-21.21,-159.78,80.60,74,89,23.04,overcast clouds
2,2,Kismayo,SO,-0.36,42.55,80.26,82,76,12.37,light rain
3,3,Rikitea,PF,-23.12,-134.97,76.23,79,100,18.28,overcast clouds
4,4,Guigue,VE,10.09,-67.78,73.02,89,97,3.49,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 5
What is the maximum temperature you would like for your trip? 10


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
17,17,Semey,KZ,50.41,80.23,5.00,84,74,6.71,broken clouds
39,39,College,US,64.86,-147.80,8.60,84,75,4.90,broken clouds
48,48,Stepnogorsk,KZ,52.35,71.89,8.40,95,100,8.12,overcast clouds
63,63,Barrow,US,71.29,-156.79,6.80,77,90,12.75,overcast clouds
211,211,Vikulovo,RU,56.82,70.61,6.42,94,93,5.84,overcast clouds
219,219,Barsovo,RU,61.17,73.17,5.00,92,75,4.47,broken clouds
239,239,Teya,RU,60.38,92.63,8.51,87,74,0.78,broken clouds
267,267,Chifeng,CN,42.27,118.96,5.27,85,0,8.28,clear sky
278,278,Cherskiy,RU,68.75,161.30,5.59,90,28,2.46,scattered clouds
353,353,Kommunisticheskiy,RU,61.68,64.48,9.46,96,33,2.89,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                15
City                   15
Country                15
Lat                    15
Lng                    15
Max Temp               15
Humidity               15
Cloudiness             15
Wind Speed             15
Current Description    15
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
17,17,Semey,KZ,50.41,80.23,5.00,84,74,6.71,broken clouds
39,39,College,US,64.86,-147.80,8.60,84,75,4.90,broken clouds
48,48,Stepnogorsk,KZ,52.35,71.89,8.40,95,100,8.12,overcast clouds
63,63,Barrow,US,71.29,-156.79,6.80,77,90,12.75,overcast clouds
211,211,Vikulovo,RU,56.82,70.61,6.42,94,93,5.84,overcast clouds
219,219,Barsovo,RU,61.17,73.17,5.00,92,75,4.47,broken clouds
239,239,Teya,RU,60.38,92.63,8.51,87,74,0.78,broken clouds
267,267,Chifeng,CN,42.27,118.96,5.27,85,0,8.28,clear sky
278,278,Cherskiy,RU,68.75,161.30,5.59,90,28,2.46,scattered clouds
353,353,Kommunisticheskiy,RU,61.68,64.48,9.46,96,33,2.89,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Semey,KZ,5.00,broken clouds,50.41,80.23,
39,College,US,8.60,broken clouds,64.86,-147.80,
48,Stepnogorsk,KZ,8.40,overcast clouds,52.35,71.89,
63,Barrow,US,6.80,overcast clouds,71.29,-156.79,
211,Vikulovo,RU,6.42,overcast clouds,56.82,70.61,
219,Barsovo,RU,5.00,broken clouds,61.17,73.17,
239,Teya,RU,8.51,broken clouds,60.38,92.63,
267,Chifeng,CN,5.27,clear sky,42.27,118.96,
278,Cherskiy,RU,5.59,scattered clouds,68.75,161.30,
353,Kommunisticheskiy,RU,9.46,scattered clouds,61.68,64.48,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(5)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Semey,KZ,5.00,broken clouds,50.41,80.23,Hotel Zhana-Tau
39,College,US,8.60,broken clouds,64.86,-147.80,Pike's Waterfront Lodge
48,Stepnogorsk,KZ,8.40,overcast clouds,52.35,71.89,Gostinitsa Stepnogorsk
63,Barrow,US,6.80,overcast clouds,71.29,-156.79,King Eider Inn
211,Vikulovo,RU,6.42,overcast clouds,56.82,70.61,"Ooo ""Sistema Stroy"""


In [10]:
# 8a. Create the output File (CSV)
output_hotel_file = r"C:\Users\ancya\Documents\Weather_Database\WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_hotel_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
gmaps.configure(api_key = g_key)
locations = clean_hotel_df[['Lat','Lng']]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [13]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))